<h2>Lab2: Spark SQL</h2>

In [ ]:
import os
os.environ['SPARK_HOME'] = "/home/zaranik/.sdkman/candidates/spark/current"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python3'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Ініціалізація SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName("FrogDataAnalysis") \
    .getOrCreate()
spark

Завантаження схеми даних

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType

frog_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("pres.abs", IntegerType(), False),
    StructField("northing", IntegerType(), False),
    StructField("easting", IntegerType(), False),
    StructField("altitude", IntegerType(), False),
    StructField("distance", IntegerType(), False),
    StructField("NoOfPools", IntegerType(), False),
    StructField("NoOfSites", IntegerType(), False),
    StructField("avrain", DoubleType(), False),
    StructField("meanmin", DoubleType(), False),
    StructField("meanmax", DoubleType(), False)
])

df = spark.read.csv("./frogs.csv", header=True, schema=frog_schema)
df = df.withColumnRenamed("pres.abs", "presence")

Створення таблиці та вивід перших 5 рядків на екран

In [ ]:
df.createOrReplaceTempView("Frogs")
spark.sql("SELECT * from Frogs").show(5)

Завдання 1: Створення нової таблиці з двома стовпчиками: distance та массив NoOfPools

Синтаксис Spark SQL дещо відрізняється від стандартного SQl, оскільки має набір функій для роботи з массивами: collect_list, array_distinct, array_max та інші.

In [ ]:
df_task1 = spark.sql("""
    SELECT distance, collect_list(NoOfPools) AS NoOfPools
    FROM Frogs
    GROUP BY distance
    ORDER BY distance
""")

Створення таблиці та виведення її

In [ ]:
df_task1.createOrReplaceTempView("Task1")
spark.sql("SELECT * from Task1").show()

Завдання 2: Взяти таблицю з завдання 1 і перетворити другий стовпець на кілька
стовпців.

Знаходження максимальної довжини масиву, вона потрібна підрахунку кількості колонок у новй тимчасовій таблиці

In [ ]:
max_length = df_task1.select(F.size("NoOfPools")).agg(F.max("size(NoOfPools)")).collect()[0][0]

Створюємо тимчасову таблицю, розвертаючи массив на кілька колонок

In [ ]:
columns = ", ".join([f"NoOfPools[{i}] AS Pool_{i}" for i in range(max_length)])
df_task2 = spark.sql(f"""
    SELECT distance, {columns}
    FROM Task1
""")

Створення таблиці та виведення її

In [ ]:
df_task2.createOrReplaceTempView("Task2")
spark.sql("SELECT * from Task2").show()

Завдання 3: Знайти мінімальну висоту для усіх можливих поєднань параметрів
pres.abs та NoOfSites.

Достатньо простих запит на group by, що викоритсовує функцію MIN по полю altitude

In [ ]:
df_task3 = spark.sql("""
    SELECT presence, NoOfSites, MIN(altitude) AS min_altitude
    FROM Frogs
    GROUP BY presence, NoOfSites
""")
df_task3.createOrReplaceTempView("Task3")
spark.sql("SELECT * from Task3").show()

Завдання 4: Додати в таблицю frogs новий стовпець, що містить різницю між
максимальною та поточною відстанню до поселення в залежності
від того чи спостерігались жаби. Наприклад, якщо максимальна до
поселення від місця, де спостерігались жаби 1000, а в першому
рядку відстань 800 і жаби спостерігались, то значення в
додатковому стовпчику = 1000-800=200. Зберегти новий файл.

Визначаємо максимальне значення distance для жаб 

In [ ]:
max_distance = df.where("presence = 1").agg(F.max("distance")).collect()[0][0]

Додаємо новий стовпчик з різницею між
максимальною та поточною відстанню до поселення в залежності
від того чи спостерігались жаби. За умовой задачі якщо presence == 0, то жаби спостерігаються, а якщо 1 - то ні

In [ ]:
df_task4 = spark.sql(f"""
    SELECT *, 
           CASE WHEN presence = 0 THEN {max_distance} - distance ELSE NULL END AS distance_diff
    FROM Frogs
""")
df_task4.createOrReplaceTempView("Task4")
spark.sql("SELECT * from Task4").show()

Збереження кінцевої таблиці до файлу

In [ ]:
df_task4.write.csv("./modified_frogs.csv", header=True)

In [ ]:
spark.stop()